In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.basics import *

In [ ]:
#export
class TypeDispatch2(TypeDispatch):
    def __call__(self, *args, **kwargs):
        res = super().__call__(*args, **kwargs)
        if res is args[0]: return None
        return res

In [ ]:
#export
class DispatchReg2(DispatchReg):
    def __init__(self): self.d = defaultdict(TypeDispatch2)
typedispatch2 = DispatchReg2()

In [ ]:
#export
def reduce_lbls(lbls):
    lbls = lbls.filter(partial(equals, None), negate=True) # Remove all Nones
    if len(lbls)>1: raise ValueError # TODO: Should figure out this case
    return categorize(lbls[0] if len(lbls)==1 else ABSTAIN)

In [ ]:
#export
def compose_tfms2(x, tfms, lfs, is_enc=True, reverse=False, **kwargs):
    "Apply all `func_nm` attribute of `tfms` on `x`, maybe in `reverse` order"
    if reverse: tfms = reversed(tfms)
    lbls = L(L() for _ in range_of(lfs))
    for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    for f in tfms:
        if not is_enc: f = f.decode
        x = f(x, **kwargs)
        for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    # TODO: LF can be called twice (if types repeat)
    lbls = [reduce_lbls(v) for v in lbls]
    return lbls

In [ ]:
#export
@delegates(Pipeline.__init__)
class Pipeline2(Pipeline):
    def __init__(self, funcs=None, lfs=None, **kwargs):
        super().__init__(funcs=funcs, **kwargs)
        self.lfs = lfs
    def __call__(self, o): return compose_tfms2(o, tfms=self.fs, lfs=self.lfs,
                                                split_idx=self.split_idx)

In [ ]:
#export
ABSTAIN = 'abstain'
categorize = Categorize(vocab=[ABSTAIN])
# TODO: Order can change if incrementally add new items
def register_categories(*args):
    global categorize
    categorize = Categorize(vocab=itertools.chain(categorize.vocab,args))

Tests

In [ ]:
CAT1,CAT2 = 'CAT1','CAT2'
register_categories(CAT1, CAT2)

In [ ]:
@typedispatch2
def fint(x:int): return CAT1
@typedispatch2
def fstr(x:str): return CAT2
@typedispatch2
def fflt(x:float): return CAT2
@typedispatch2
def flst(x:list): return CAT1

In [ ]:
@typedispatch
def flt2int(x:float): return int(x)
@typedispatch
def int2str(x:int): return str(x)
@typedispatch
def int2flt(x:int): return float(x)
@typedispatch
def flt2str(x:float): return str(x)
@typedispatch
def str2int(x:str): return int(float(x))

In [ ]:
tfms = Pipeline2([flt2str, str2int], lfs=[fint, fstr, fflt, flst])
test_eq(tfms(1.2), map(categorize, [CAT1,CAT2,CAT2,ABSTAIN]))

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted Untitled-Copy1.ipynb.
Converted index.ipynb.
Converted resume.ipynb.
